In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.special import boxcox1p
import pickle
import h5py
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import joblib
import base64

In [34]:
df = pd.read_csv('PREMIER LEAGUE.csv')

In [35]:
selected_columns = ['FTR','HS(Home Team Shots)','AS(Away Team Shots)','HST(Home Team Shots on Target)','AST(Away Team Shots on Target)','Team1_FormPoints','Team2_FormPoints']
correlation_matrix = df[selected_columns].corr()
important_correlations = correlation_matrix[abs(correlation_matrix) > 0.4]
print(important_correlations)

                                HS(Home Team Shots)  AS(Away Team Shots)  \
HS(Home Team Shots)                        1.000000            -0.413444   
AS(Away Team Shots)                       -0.413444             1.000000   
HST(Home Team Shots on Target)             0.651684                  NaN   
AST(Away Team Shots on Target)                  NaN             0.659700   
Team1_FormPoints                                NaN                  NaN   
Team2_FormPoints                                NaN                  NaN   

                                HST(Home Team Shots on Target)  \
HS(Home Team Shots)                                   0.651684   
AS(Away Team Shots)                                        NaN   
HST(Home Team Shots on Target)                        1.000000   
AST(Away Team Shots on Target)                             NaN   
Team1_FormPoints                                           NaN   
Team2_FormPoints                                           NaN   

    

<ipython-input-35-ff9904df03a9>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df[selected_columns].corr()


In [36]:
from scipy.stats import chi2_contingency

In [37]:
categorical_columns = ['FTR','HS(Home Team Shots)','AS(Away Team Shots)','HST(Home Team Shots on Target)','AST(Away Team Shots on Target)','Team1_FormPoints','Team2_FormPoints']
contingency_table = pd.crosstab(df[categorical_columns[0]], df[categorical_columns[1]])
chi2, p, _, _ = chi2_contingency(contingency_table)
chi2

396.18243604463515

In [38]:
df.head()

,Div,Date,HomeTeam,AwayTeam,FTR,HS(Home Team Shots),AS(Away Team Shots),HST(Home Team Shots on Target),AST(Away Team Shots on Target),Team1_FormPoints,Team2_FormPoints
0,E0,14-08-2010,Aston Villa,West Ham,H,23,12,11,2,4,0
1,E0,14-08-2010,Blackburn,Everton,H,7,17,2,12,4,1
2,E0,14-08-2010,Bolton,Fulham,D,13,12,9,7,7,5
3,E0,14-08-2010,Chelsea,West Brom,H,18,10,13,4,10,10
4,E0,14-08-2010,Sunderland,Birmingham,D,6,13,2,7,4,5


In [39]:
df.shape

(5060, 11)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5060 entries, 0 to 5059
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Div                             5060 non-null   object
 1   Date                            5060 non-null   object
 2   HomeTeam                        5060 non-null   object
 3   AwayTeam                        5060 non-null   object
 4   FTR                             5060 non-null   object
 5   HS(Home Team Shots)             5060 non-null   int64 
 6   AS(Away Team Shots)             5060 non-null   int64 
 7   HST(Home Team Shots on Target)  5060 non-null   int64 
 8   AST(Away Team Shots on Target)  5060 non-null   int64 
 9   Team1_FormPoints                5060 non-null   int64 
 10  Team2_FormPoints                5060 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 435.0+ KB


In [41]:
df.describe()

,HS(Home Team Shots),AS(Away Team Shots),HST(Home Team Shots on Target),AST(Away Team Shots on Target),Team1_FormPoints,Team2_FormPoints
count,5060.000000,5060.000000,5060.000000,5060.000000,5060.000000,5060.000000
mean,14.012846,11.356522,5.460672,4.442885,6.856522,6.801779
std,5.541059,4.842138,3.178214,2.716727,3.466151,3.434076
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.000000,8.000000,3.000000,2.000000,4.000000,4.000000
50%,14.000000,11.000000,5.000000,4.000000,7.000000,7.000000
75%,17.000000,14.000000,7.000000,6.000000,9.000000,9.000000
max,43.000000,31.000000,24.000000,20.000000,15.000000,15.000000


In [42]:
df.isnull().any()

Div                               False
Date                              False
HomeTeam                          False
AwayTeam                          False
FTR                               False
HS(Home Team Shots)               False
AS(Away Team Shots)               False
HST(Home Team Shots on Target)    False
AST(Away Team Shots on Target)    False
Team1_FormPoints                  False
Team2_FormPoints                  False
dtype: bool

In [43]:
df.isnull().sum() * 100 / len(df)

Div                               0.0
Date                              0.0
HomeTeam                          0.0
AwayTeam                          0.0
FTR                               0.0
HS(Home Team Shots)               0.0
AS(Away Team Shots)               0.0
HST(Home Team Shots on Target)    0.0
AST(Away Team Shots on Target)    0.0
Team1_FormPoints                  0.0
Team2_FormPoints                  0.0
dtype: float64

In [44]:
df = df.fillna(df.median())

<ipython-input-44-a187aa03e3ee>:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.median())


In [45]:
columns_to_check = ['FTR','HS(Home Team Shots)','AS(Away Team Shots)','HST(Home Team Shots on Target)','AST(Away Team Shots on Target)','Team1_FormPoints','Team2_FormPoints']
duplicates = df[df.duplicated(subset=columns_to_check, keep=False)]
print("Rows with duplicates in specific columns:")
print(duplicates)

Rows with duplicates in specific columns:
     Div        Date     HomeTeam      AwayTeam FTR  HS(Home Team Shots)  \
493   E0  19-11-2011   Sunderland        Fulham   D                   13   
890   E0  27-11-2012   Sunderland           QPR   D                   13   
1478  E0  19-04-2014      Cardiff         Stoke   D                   13   
2714  E0  23-09-2017  Southampton    Man United   A                   14   
2975  E0  07-04-2018     Brighton  Huddersfield   D                   13   
4179  E0  23-05-2021       Wolves    Man United   A                   14   

      AS(Away Team Shots)  HST(Home Team Shots on Target)  \
493                    13                               8   
890                    13                               8   
1478                   14                               4   
2714                    9                               4   
2975                   14                               4   
4179                    9                               4  

In [46]:
def outlier(dataframe,col_name,q1=0.25,q3=0.75):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    inter_quartile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * inter_quartile_range
    low_limit = quartile1 -  1.5 * inter_quartile_range
    return low_limit,up_limit


def check_outlier(dataframe,col_name):
    low_limit,up_limit = outlier(dataframe,col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

def detect_outliers(dataframe,col_name,return_index=False):
    low,up = outlier(dataframe,col_name)
    if dataframe[((dataframe[col_name]<low) | (dataframe[col_name]>up))].shape[0] > 10:
        print(dataframe[((dataframe[col_name]<low) | (dataframe[col_name]>up))].head())
    else:
        print(dataframe[((dataframe[col_name]<low) | (dataframe[col_name]> up))])

    if return_index:
        outlier_index = dataframe[((dataframe[col_name]< low) | (dataframe[col_name]>up))].index
        return outlier_index


def fill_na_with_mean(df):
    num_cols = df.select_dtypes(include = ["float64","int64"]).columns
    for col in num_cols:
        df[col].fillna(df[col].mean(),inplace=True)
    return df

def fill_na_with_median(df):
    num_cols = df.select_dtypes(include = ["float64","int64"]).columns
    for col in num_cols:
        df[col].fillna(df[col].median(),inplace=True)
    return df

def fill_na_with_o(df):
    return df.fillna(0)

def fil_na_with_mode(df):
    for col in cat_cols:
        mode_val = df[col].mode()[0]
        df[col].fillna(mode_val,inplace=True)
    return df

In [47]:
def get_columns(dataframe,cat_threshold=10):
    num_but_cat_cols = [col for col in dataframe.columns if dataframe[col].nunique()<cat_threshold and
                        dataframe[col].dtype!="O"]


    #numerical columns
    num_cols = [col for col in dataframe.columns if dataframe[col].dtype!="O"]
    num_cols = [col for col in num_cols if col not in num_but_cat_cols]

    print(f"Number of observations: {dataframe.shape[0]}")
    print(f"Number of Variables: {dataframe.shape[1]}")
    #print(f"Number of categorical_columns:{len(cat_cols)}")
    print(f"Number of numerical_columns treated as categorical: {len(num_but_cat_cols)}")
    return num_cols

num_cols = get_columns(df)

Number of observations: 5060
Number of Variables: 11
Number of numerical_columns treated as categorical: 0


In [48]:
def remove_outlier(dataframe,col_name):
    low_limit,up_limit = outlier(dataframe,col_name)
    df_with_out_outliers = dataframe[~((dataframe[col_name]<low_limit) | (dataframe[col_name]>up_limit))]
    return df_with_out_outliers

In [49]:
def replace_with_max_min_limit(dataframe,variable):
    low_limit,up_limit = outlier(dataframe,variable)
    dataframe.loc[(dataframe[variable]<low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable]>up_limit),variable] = up_limit

In [50]:
for col in num_cols:
    remove_outlier(df,col).shape
    replace_with_max_min_limit(df,col)

for col in num_cols:
    fill_na_with_median(df)

In [51]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['FTR'] = label_encoder.fit_transform(df['FTR'])

In [52]:
X=df.iloc[:,5:]
X

,HS(Home Team Shots),AS(Away Team Shots),HST(Home Team Shots on Target),AST(Away Team Shots on Target),Team1_FormPoints,Team2_FormPoints
0,23.0,12,11,2,4.0,0.0
1,7.0,17,2,12,4.0,1.0
2,13.0,12,9,7,7.0,5.0
3,18.0,10,13,4,10.0,10.0
4,6.0,13,2,7,4.0,5.0
...,...,...,...,...,...,...
5055,12.0,9,6,5,5.0,6.0
5056,11.0,9,6,1,4.0,7.0
5057,17.0,16,10,3,10.0,7.0
5058,16.0,10,6,5,4.0,4.0


In [53]:
y=df.FTR
y

0       2
1       2
2       1
3       2
4       1
       ..
5055    2
5056    1
5057    2
5058    2
5059    1
Name: FTR, Length: 5060, dtype: int64

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5060 entries, 0 to 5059
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Div                             5060 non-null   object 
 1   Date                            5060 non-null   object 
 2   HomeTeam                        5060 non-null   object 
 3   AwayTeam                        5060 non-null   object 
 4   FTR                             5060 non-null   int64  
 5   HS(Home Team Shots)             5060 non-null   float64
 6   AS(Away Team Shots)             5060 non-null   int64  
 7   HST(Home Team Shots on Target)  5060 non-null   int64  
 8   AST(Away Team Shots on Target)  5060 non-null   int64  
 9   Team1_FormPoints                5060 non-null   float64
 10  Team2_FormPoints                5060 non-null   float64
dtypes: float64(3), int64(4), object(4)
memory usage: 435.0+ KB


In [55]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Manually chosen hyperparameters
best_params = {
   'learning_rate': 2,  # Slightly higher learning rate
    'n_estimators': 50,  # Slightly fewer trees
    'max_depth': 1,
    'min_child_weight': 1,
    'subsample': 0.85,  # Slightly lower subsample
    'colsample_bytree': 1,
    'gamma': 4,
    'reg_alpha': 0.2,  # Slightly higher reg_alpha
    'reg_lambda': 0, # Slightly higher reg_lambda
    'max_delta_step': 1,
    #'colsample_bylevel': 1,
    #'colsample_bynode': 1
}

best_model = xgb.XGBClassifier(**best_params, tree_method='hist')
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Final Accuracy: %.2f%%" % (accuracy * 100.0))


Final Accuracy: 61.17%


In [59]:
df1 = pd.read_excel('OCTOBER DATA.xlsx')
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df1['FTR'] = label_encoder.fit_transform(df1['FTR'])
FTR = df1['FTR']
features = df1.drop('FTR', axis=1)
pred_prob = best_model.predict_proba(features)
predictions = np.argmax(pred_prob, axis=1)
pred_draw = pred_prob[:,1]
pred_lose = pred_prob[:,0]
pred_win = pred_prob[:,2]
df1 = pd.DataFrame({'Predicted': predictions, 'Probability Loss': pred_lose, 'Probability Draw': pred_draw, 'Probability Win': pred_win})
decoded_data = label_encoder.inverse_transform(df1['Predicted'])
df2 = pd.DataFrame({'Predicted': decoded_data, 'Probability Loss': pred_lose, 'Probability Draw': pred_draw, 'Probability Win': pred_win})
df2.to_excel('Predicted Data.xlsx')
accuracy = accuracy_score(FTR, predictions)
print("Final Accuracy: %.2f%%" % (accuracy * 100.0))

Final Accuracy: 63.64%
